# Imports

In [1]:
import pandas as pd
import osmnx as ox # https://towardsdatascience.com/visualization-in-python-finding-routes-between-points-2d97d4881996
import networkx as nx
# from geopy.geocoders import Nominatim

ox.settings.log_console=True
ox.settings.use_cache=True

# Data

In [2]:
data = pd.read_csv('../data/alt_fuel_stations_Feb_2_2023.csv')
data.head()

/var/folders/sy/b9nnsg_n78145yg6bd0j_hz00000gn/T/ipykernel_54684/1397583120.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/alt_fuel_stations_Feb_2_2023.csv')


,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access,RD Blends,RD Blends (French),RD Blended with Biodiesel,RD Maximum Biodiesel Level
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,West hall and South hall,Los Angeles,CA,90015,NaN,213-741-1151,E,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
1,ELEC,California Air Resources Board,9530 Telstar Ave,NaN,El Monte,CA,91731,NaN,626-575-6800,E,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2,ELEC,Scripps Green Hospital,10666 N Torrey Pines Rd,"Patient Parking Structure, level G",La Jolla,CA,92037,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
3,ELEC,Galpin Motors,15421 Roscoe Blvd,NaN,Sepulveda,CA,91343,NaN,800-256-6219,E,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
4,ELEC,Galleria at Tyler,1299 Galleria at Tyler,NaN,Riverside,CA,92503,NaN,951-351-3110,E,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


# Methodology

# 

# Testing

Start: USC (34.022549259439856, -118.28512300406169)

End: Calabasas (34.150624732296876, -118.69563423061471)

<!-- End: San Francisco (37.76287289552181, -122.43505673838006) -->

In [5]:
start = (34.022549259439856, -118.28512300406169)
end = (34.150624732296876, -118.69563423061471)
# end = (37.76287289552181, -122.43505673838006)

place = 'Los Angeles, California, United States'
mode = 'drive'
optimizer = 'length'

In [6]:
graph = ox.graph_from_place(place, network_type=mode)

orig_node = ox.distance.nearest_nodes(graph, start[1], start[0])
dest_node = ox.distance.nearest_nodes(graph, end[1], end[0])

shortest_route = nx.shortest_path(graph, orig_node, dest_node, weight=optimizer)

In [7]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route)
shortest_route_map

# Google Maps api

# References
https://pypi.org/project/googlemaps/